In [2]:
import os
import sys
import warnings
import pandas as pd
from tqdm.notebook import tqdm
import pickle
from socceraction.data.statsbomb import StatsBombLoader
from statsbombpy import sb

base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(f"base_path: {base_path}")
sys.path.append(base_path)
from express.databases import SQLiteDatabase
from express.visualization import plot_action
from express.utils import add_names
from functools import partial

from express.datasets import PressingDataset
from tqdm import tqdm
import pandas as pd
from pathlib import Path

import shap
import matplotlib.pyplot as plt


base_path: c:\Users\mirun\24_winter\press\sr-press


In [34]:
model = "xgboost" #"soccermap"
trial = 10 # as you like

path = Path(base_path) / "stores" / "model" / model / f"{trial:03d}" / "component.pkl"
with path.open(mode="rb") as file:
    component = pickle.load(file)
component

In [3]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

SBL = StatsBombLoader(getter="remote", creds={"user": None, "passwd": None})
TEST_DB_PATH = os.path.join(base_path, "stores/test_database.sqlite")
test_db = SQLiteDatabase(TEST_DB_PATH)
dataset_test = partial(PressingDataset, path=os.path.join(base_path, "stores", "filtered_datasets", "test"))


In [35]:
component.features

{'relative_startlocation': ['start_dist_sideline_a0',
  'start_dist_goalline_a0',
  'start_dist_sideline_a1',
  'start_dist_goalline_a1',
  'start_dist_sideline_a2',
  'start_dist_goalline_a2'],
 'dist_opponent': ['dist_defender_start_a0',
  'dist_defender_end_a0',
  'dist_defender_action_a0',
  'dist_defender_start_a1',
  'dist_defender_end_a1',
  'dist_defender_action_a1',
  'dist_defender_start_a2',
  'dist_defender_end_a2',
  'dist_defender_action_a2'],
 'closest_11_players': ['teammate_1_x_a0',
  'teammate_1_y_a0',
  'teammate_1_distance_a0',
  'teammate_2_x_a0',
  'teammate_2_y_a0',
  'teammate_2_distance_a0',
  'teammate_3_x_a0',
  'teammate_3_y_a0',
  'teammate_3_distance_a0',
  'teammate_4_x_a0',
  'teammate_4_y_a0',
  'teammate_4_distance_a0',
  'teammate_5_x_a0',
  'teammate_5_y_a0',
  'teammate_5_distance_a0',
  'teammate_6_x_a0',
  'teammate_6_y_a0',
  'teammate_6_distance_a0',
  'teammate_7_x_a0',
  'teammate_7_y_a0',
  'teammate_7_distance_a0',
  'teammate_8_x_a0',
  'te

In [36]:
preds = component.predict(dataset_test)
preds

game_id  action_id
3857254  4            0.248037
         14           0.121798
         16           0.110283
         20           0.283783
         32           0.226375
                        ...   
3869685  2914         0.235998
         2920         0.584305
         2931         0.385363
         2938         0.481119
         2941         0.460043
Length: 14504, dtype: float32

In [37]:
test_dataset = PressingDataset(
    path= os.path.join(base_path, "stores", "filtered_datasets", "test"),
    xfns=["relative_startlocation", "dist_opponent", "closest_11_players", "result", "get_average_column_sum"],
    yfns=["possession_change_by_5_actions"],
    load_cached=True,
    nb_prev_actions=3
)

In [38]:
new=test_dataset.labels.copy()
new['posession_change_prob'] = preds.values
new


possession_change_by_5_actions  posession_change_prob
game_id action_id                                                       
3857254 4                                   False               0.248037
        14                                  False               0.121798
        16                                  False               0.110283
        20                                  False               0.283783
        32                                  False               0.226375
...                                           ...                    ...
3869685 2914                                False               0.235998
        2920                                 True               0.584305
        2931                                False               0.385363
        2938                                False               0.481119
        2941                                False               0.460043

[14504 rows x 2 columns]

### pressing action마다 Player_id 찾아서 append하기
- 이것보다 빠르게 구할 수 있는 방법 아신다면 수정해주십시오...
- 50분정도 걸립니다

### 압박 및 탈압박 선수 id 구하기

In [17]:
# game_id 인덱스를 리스트로 변환
game_id_list = test_db.games().index.unique().tolist()

# 리스트 확인
print(game_id_list)


[3857254, 3857255, 3857256, 3857257, 3857258, 3857259, 3857260, 3857261, 3857262, 3857263, 3857264, 3857265, 3857266, 3857267, 3857268, 3857269, 3857270, 3857271, 3857272, 3857273, 3857274, 3857275, 3857276, 3857277, 3857278, 3857279, 3857280, 3857281, 3857282, 3857283, 3857284, 3857285, 3857286, 3857287, 3857288, 3857289, 3857290, 3857291, 3857292, 3857293, 3857294, 3857295, 3857296, 3857297, 3857298, 3857299, 3857300, 3857301, 3869117, 3869118, 3869151, 3869152, 3869219, 3869220, 3869253, 3869254, 3869321, 3869354, 3869420, 3869486, 3869519, 3869552, 3869684, 3869685]


In [27]:
from tqdm import tqdm
import pandas as pd

# 결과 저장용 리스트
states = []
pressure_states = []
pressured_player_ids = []
pressing_action_ids = []
game_ids = []  # game_id 저장용 리스트 추가

# 테스트용 데이터 로드
for game_id in tqdm(test_db.games().index.unique().tolist(), desc="Processing games"):
    actions = add_names(test_db.actions(game_id)).reset_index()

    # 필터 초기화
    idx = pd.Series([True] * len(actions), index=actions.index)

    if idx.sum() < 1:
        print(f"No actions available for game_id {game_id}")
        continue

    # 홈팀 정보 가져오기
    home_team_id, _ = test_db.get_home_away_team_id(game_id)

    # Group by period and process
    for (game_id, period_id), group in tqdm(
        actions.groupby(["game_id", "period_id"], sort=False),
        total=len(actions["period_id"].unique()),
        desc=f"Processing periods for game_id {game_id}"
    ):
        # 액션 반복
        for action in group.itertuples(index=False):
            if action.type_name == "pressing": 
                continue

            start_time = action.time_seconds
            end_time = start_time + action.duration
            team_id = action.team_id

            # Overlap 조건 설정
            overlap_cond1 = (actions['time_seconds'] <= start_time) & ((actions['time_seconds'] + actions['duration']) >= start_time)
            overlap_cond2 = (actions['time_seconds'] >= start_time) & ((actions['time_seconds'] + actions['duration']) <= end_time)
            overlap = (
                (actions["game_id"] == game_id) &
                (actions["period_id"] == period_id) &
                (actions["team_id"] != team_id) &  # 팀 비교
                overlap_cond1
            )

            # 압박 액션 필터링
            pressing_actions = actions[overlap & idx]

            # 데이터 저장
            for pressing_action in pressing_actions.itertuples(index=False):       
                states.append(action)
                pressure_states.append(pressing_action)
                pressured_player_ids.append(action.player_id)
                pressing_action_ids.append(pressing_action.action_id)
                game_ids.append(game_id)  # game_id 추가

# 압박 데이터 확인
print(f"Number of pressing actions: {len(pressing_action_ids)}")
print(f"Number of pressured players: {len(pressured_player_ids)}")

# 데이터를 DataFrame으로 변환
result_df = pd.DataFrame({
    "game_id": game_ids,  # game_id 컬럼 추가
    "pressing_action_id": pressing_action_ids,
    "pressured_player_id": pressured_player_ids
})

# 결과 확인
print(result_df.head())


Processing games: 100%|██████████| 64/64 [06:10<00:00,  5.79s/it]

Number of pressing actions: 20359
Number of pressured players: 20359
   game_id  pressing_action_id  pressured_player_id
0  3857254                   4                17042
1  3857254                   7                 5651
2  3857254                  14                16554
3  3857254                  16                 3570
4  3857254                  16                 3570


In [33]:
filtered_pressure_states = [
    state for state in pressure_states if state.type_name == "pressing"
]

print(f"Number of pressing states: {len(filtered_pressure_states)}")
for state in filtered_pressure_states[:5]:  # 상위 5개 요소 출력
    print(state)


Number of pressing states: 11314
Pandas(game_id=3857254, action_id=4, original_event_id='5af23d18-48fa-4708-8df8-02134574b7de', period_id=1, time_seconds=3.176, team_id=777, player_id=23910, start_x=69.3875, start_y=23.799999999999997, end_x=69.3875, end_y=23.799999999999997, bodypart_id=0, type_id=23, result_id=0, possession_team_id=776.0, play_pattern_name='From Kick Off', under_pressure=0, duration=0.695959, extra={}, visible_area_360=[[99.8327620665036, 68.0], [71.38467021012661, 5.436102909760308], [53.628566958584976, 7.659681141203841], [43.10874763376633, 68.0], [99.8327620665036, 68.0]], in_visible_area_360=1, freeze_frame_360=[{'teammate': True, 'actor': False, 'keeper': False, 'x': 83.13400164556944, 'y': 34.876250831734225}, {'teammate': True, 'actor': False, 'keeper': False, 'x': 80.16366202477967, 'y': 25.76470170626603}, {'teammate': True, 'actor': False, 'keeper': False, 'x': 79.24324500709986, 'y': 46.070024350537565}, {'teammate': True, 'actor': False, 'keeper': False

In [28]:
len(pressing_action_ids)

20359

In [24]:
len(pressured_player_ids)

20359

### player_name 매핑해서 append하기

In [39]:
result_df.head(10)

,game_id,pressing_action_id,pressured_player_id
0,3857254,4,17042
1,3857254,7,5651
2,3857254,14,16554
3,3857254,16,3570
4,3857254,16,3570
5,3857254,19,44955
6,3857254,23,17042
7,3857254,23,17042
8,3857254,32,5651
9,3857254,35,5655


In [40]:
new.head(10)

possession_change_by_5_actions  posession_change_prob
game_id action_id                                                       
3857254 4                                   False               0.248037
        14                                  False               0.121798
        16                                  False               0.110283
        20                                  False               0.283783
        32                                  False               0.226375
        35                                  False               0.331838
        37                                  False               0.565257
        42                                  False               0.333415
        51                                  False               0.222846
        86                                  False               0.353204

In [ ]:
import json
with open('teams_2_id.json', 'r', encoding='utf-8') as file:
    team_mapping = json.load(file)
new['team_name'] = new['team_id'].astype(str).map(team_mapping)
new['xP_difference'] =  new['possession_change_by_seconds'] - new['posession_change_prob']
new


In [ ]:
# 1. 팀당 pressing point 합 구하기
team_sum = new.groupby('team_name')['posession_change_prob'].sum().reset_index()

# 2. 팀당 xP_difference 합 구하기
team_difference = new.groupby('team_name')['xP_difference'].sum().reset_index()

# 3. 팀당 서로 다른 game_id 개수 세기
team_game_count = new.reset_index().groupby('team_name')['game_id'].nunique()
team_game_count_df = team_game_count.reset_index()


# 4. 팀당 서로 다른 action_id 개수 세기
team_action_count = new.reset_index().groupby('team_name')['action_id'].nunique()
team_action_count= team_action_count.reset_index()

# 데이터프레임 병합

merged_df = pd.merge(team_sum, team_difference, on='team_name', how='outer')  # 첫 번째 병합
merged_df = pd.merge(merged_df, team_game_count, on='team_name', how='outer')  # 두 번째 병합
merged_df = pd.merge(merged_df, team_action_count, on='team_name', how='outer')  # 세 번째 병합
merged_df.rename(columns={'game_id': 'games_played'}, inplace=True)
merged_df.rename(columns={'action_id': 'actions'}, inplace=True)
merged_df.rename(columns={'posession_change_prob': 'xP_total'}, inplace=True)

# 추가 컬럼 계산
merged_df['xP_total']=merged_df['xP_total'].round(2)
merged_df['xP_difference']=merged_df['xP_difference'].round(2)
merged_df['xP_per_game'] = (merged_df['xP_total'] / merged_df['games_played']).round(2)
merged_df['xP_per_action'] = (merged_df['xP_total'] / merged_df['actions']).round(2)


column_order = [
    'team_name',               # 팀 이름
    'games_played',            # 총 게임 수
    'actions',                 # 총 action_id 수    
    'xP_total',                # 팀당 pressing point 합
    'xP_difference',           # xP_difference
    
    'xP_per_game',             # 경기당 xP
    'xP_per_action'            # action당 xP
]

# 열 순서를 재배치
merged_df = merged_df[column_order]

# 정렬
merged_df.sort_values('xP_difference', ascending=False, inplace=True)

merged_df